В этом задании вам предлагается начать разбираться с задачей анализа тональности отзывов на примере сентимент-анализа отзывов на фильмы.

Мы будем использовать стандартный датасет из nltk, уже возникавший в одном из примеров в предыдущих курсах. Для того, чтобы импортировать необходимый модуль, напишите:

In [1]:
from nltk.corpus import movie_reviews
import numpy as np


Чтобы получить id-шники негативных и позитивных отзывов:

In [2]:
negids = movie_reviews.fileids('neg')
posids = movie_reviews.fileids('pos')

print(negids[:5])

['neg/cv000_29416.txt', 'neg/cv001_19502.txt', 'neg/cv002_17424.txt', 'neg/cv003_12683.txt', 'neg/cv004_12641.txt']


Чтобы получить список негативных отзывов:

In [3]:
negfeats = [movie_reviews.words(fileids=[f]) for f in negids]

In [4]:
negfeats[0]

['plot', ':', 'two', 'teen', 'couples', 'go', 'to', ...]

Создайте список из текстов всех имеющихся отзывов, а также список с классами, которые будет использовать ваш классификатор - 0 для негативных отзывов и 1 для позитивных.

In [5]:
posfeats = [movie_reviews.words(fileids=[f]) for f in posids]

In [6]:
movie_reviews.words(fileids=posids[0])

['films', 'adapted', 'from', 'comic', 'books', 'have', ...]

In [7]:
texts = negfeats + posfeats
labels = [0] * len(negfeats) + [1] * len(posfeats)

In [8]:
labels[-1]

1

Подсчитайте количество отзывов в выборке

In [9]:
len(texts)

2000

Подсчитайте долю класса 1 в выборке

In [10]:
len(posfeats)/len(texts)*100

50.0

Импортируйте CountVectorizer из sklearn.feature_extraction.text. Попробуйте использовать его с настройками по умолчанию для того, чтобы получить признаковое представление каждого текста. Скорее всего, попытка не увенчается успехом. Разберитесь, в чем причина, и добейтесь того, чтобы метод fit_transform у CountVectorizer успешно отрабатывал. Подсчитайте количество признаков в CountVectorizer. Никакой предварительной обработки текста (удаление стоп-слов, нормализация слов) на этом шаге делать не надо, в качестве признаков должны использоваться частоты слов.

In [11]:
from sklearn.feature_extraction.text import CountVectorizer

In [12]:
vectorizer = CountVectorizer()

In [13]:
vectorizer.fit_transform(texts)

AttributeError: 'StreamBackedCorpusView' object has no attribute 'lower'

**приведем каждый текст в строку**

In [26]:
texts[0]

['plot', ':', 'two', 'teen', 'couples', 'go', 'to', ...]

In [27]:
new_texts = [" ".join(texts[i]) for i in range(len(texts))]

In [28]:
X = vectorizer.fit_transform(new_texts)

In [29]:
X.shape

(2000, 39659)

Соберите pipeline из CountVectorizer и LogisticRegression c настройками по-умолчанию и с помощью cross_val_score (также со стандартными настройками) оцените получаемое "из коробки" качество по accuracy

In [30]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
import warnings
warnings.filterwarnings('ignore')

In [31]:
pipeline = Pipeline([('count_vect', CountVectorizer()), ('log_reg', LogisticRegression(random_state=122))])

In [32]:
cross_val_score(pipeline, new_texts, labels, scoring='accuracy', cv=10).mean()

0.8445

In [33]:
cross_val_score(LogisticRegression(random_state=122), X, labels,scoring='accuracy', cv=10).mean()

0.8445

Аналогично accuracy, оцените качество по ROC AUC

In [34]:
cross_val_score(pipeline, new_texts, labels, scoring='roc_auc', cv=10).mean()

0.9189399999999999

Обучите логистическую регрессию на всей доступной вам выборке и выведите 5 наиболее важных для модели признаков (подумайте, какие именно признаки стоит считать такими). Вам могут пригодиться метод get_feature_names() или поле vocabulary_ у класса CountVectorizer

In [35]:
log = LogisticRegression()

In [36]:
log.fit(X,labels)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [37]:
result = list(zip(abs(log.coef_[0]), vectorizer.get_feature_names()))

In [38]:
result[0]

(0.011632980025645225, '00')

In [39]:
sorted(result, reverse=True)[:5]

[(0.8134354406678922, 'bad'),
 (0.640742793247156, 'unfortunately'),
 (0.6025749258275003, 'worst'),
 (0.5658206766485677, 'fun'),
 (0.5201987262700075, 'nothing')]

In [40]:
result1 = list(zip(abs(log.coef_[0]), sorted(vectorizer.vocabulary_)))

In [41]:
sorted(result1, reverse=True)[:5]

[(0.8134354406678922, 'bad'),
 (0.640742793247156, 'unfortunately'),
 (0.6025749258275003, 'worst'),
 (0.5658206766485677, 'fun'),
 (0.5201987262700075, 'nothing')]

Здесь и далее оценка качества будет выполняться с помощью cross_val_score с cv=5 и остальными параметрами по умолчанию. Оцените среднее качество ( .mean() ) и стандартное отклонение ( .std() ) по fold'ам для: а) pipeline из CountVectorizer() и LogisticRegression(), б) pipeline из TfidfVectorizer() и LogisticRegression(). В соответствующем пункте задания выпишите через пробел среднее в п. а, отклонение в п. а, среднее в п.б и отклонение в п. б

In [42]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [43]:
pipeline2 = Pipeline([('TfidfVectorizer', TfidfVectorizer()), ('log_reg', LogisticRegression(random_state=122))])

In [44]:
mean_a, mean_b = cross_val_score(pipeline, new_texts, labels, cv=5).mean() ,  cross_val_score(pipeline2, new_texts, labels, cv=5).mean() 

In [45]:
std_a, std_b  = cross_val_score(pipeline2, new_texts, labels, cv=5).std() ,  cross_val_score(pipeline2, new_texts, labels, cv=5).std() 

In [46]:
print(mean_a,std_a,mean_a,std_b)

0.8424999999999999 0.003999999999999995 0.8424999999999999 0.003999999999999995


Попробуйте задавать разные значения параметра min_df у CountVectorizer. Оцените качество вашего классификатора с min_df=10 и с min_df=50.

In [102]:
for i in [10,50]:
    pipeline = Pipeline([('count_vect', CountVectorizer(min_df = i)), ('log_reg', LogisticRegression(random_state=122))])
    pipeline2 = Pipeline([('TfidfVectorizer', TfidfVectorizer(min_df = i)), ('log_reg', LogisticRegression(random_state=122))])
    print ('min_df%s:' % i, cross_val_score(pipeline, new_texts, labels, cv=5).mean(),
          cross_val_score(pipeline2, new_texts, labels, cv=5).mean(),
          cross_val_score(pipeline, new_texts, labels, cv=5).std(),
          cross_val_score(pipeline2, new_texts, labels, cv=5).std(),)
    print( '\n')

min_df10: 0.8394999999999999 0.8254999999999999 0.014352700094407346 0.004582575694955849


min_df50: 0.8149999999999998 0.8145 0.015968719422671307 0.007648529270389162




**как видно, при увеличения числа стоимости отсева слов терминов с 10 до 50 - кач-во алг-ма уменьшилось**

Попробуйте использовать разные классификаторы после CountVectorizer. И vectorizer, и классификатор берите с параметрами по умолчанию. Сравните результаты для LogisticRegression, LinearSVC и SGDClassifier. Выпишите в ответе на соответствующий вопрос самое худшее качество из получившихся.

In [48]:
from sklearn.linear_model import SGDClassifier
from sklearn.svm import LinearSVC

In [1]:
z=[]
k=0
for i in [LogisticRegression(), LinearSVC(),  SGDClassifier()]:
    z.append ((cross_val_score(i,X,labels,cv=5).mean()))
    print (i , z[k])
    k+=1


NameError: name 'LogisticRegression' is not defined

**худший = LinearSVC / 0.8325**

Подготовьте список стоп-слов с помощью nltk.corpus.stopwords.words('english'), посмотрите на его элементы, и передайте его в соответствующий параметр CountVectorizer. В sklearn также предусмотрен свой список английских стоп-слов - для этого нужно задать соответствующий параметр равным строке 'english'. Оцените качество классификатора в одном и другом случае и выпишете сначала качество в первом варианте, затем во втором в соответствующем вопросе.

In [54]:
import nltk
from nltk.corpus import stopwords

In [55]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\unikum\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [58]:
stop = stopwords.words('english')

In [60]:
stop[:5]

['i', 'me', 'my', 'myself', 'we']

In [82]:
pipeline_stop = Pipeline([(' CountVectorizer', CountVectorizer(stop_words = stop)), 
                          ('log_reg', LogisticRegression(random_state=122))])


In [86]:
print('cross for nltk', cross_val_score(pipeline_stop, new_texts, labels, cv=5).mean())

cross for nltk 0.8400000000000001


In [87]:
pipeline_stop__out_of_box = Pipeline([(' CountVectorizer', CountVectorizer(stop_words = 'english')), 
                          ('log_reg', LogisticRegression(random_state=122))])

In [89]:
print('cross for stopwords from sklearn:', cross_val_score(pipeline_stop__out_of_box, new_texts, labels, cv=5).mean())

cross for stopwords from sklearn: 0.8365


**Кач-во при использовании стоп слов из библиотеки nltk выше**

Попробуйте в CountVectorizer добавить к словам биграммы и измерить качество модели. А затем постройте модель на частотах буквенных n-грамм c n от 3 до 5, указав соответствующее значение параметра ngram_range и параметр analyzer='char_wb'. Полученные два числа запишите через пробел в ответе на соответствующий вопрос.

In [93]:
pipeline_words_and_biograms = Pipeline([(' CountVectorizer', CountVectorizer(ngram_range=(1,2), stop_words = stop)), 
                          ('log_reg', LogisticRegression(random_state=122))])

In [94]:
print('cross for words and biograms', cross_val_score(pipeline_words_and_biograms, new_texts, labels, cv=5).mean())

cross for words and biograms 0.8474999999999999


In [100]:
pipeline_ngram_3_5 = Pipeline([(' CountVectorizer', CountVectorizer(ngram_range=(3,5), stop_words = stop, analyzer='char_wb')), 
                          ('log_reg', LogisticRegression(random_state=122))])

In [142]:
print('cross for letters ngrams(3 - 5)', cross_val_score(pipeline_ngram_3_5, new_texts, labels, cv=5).mean())

cross for letters ngrams(3 - 5) 0.8265


**кач-во классифиткатора даж выросло при n  = (1-2)**